In [426]:
from pulp import *
import numpy as np
import time
import random
import pandas as pd
import copy
import ujson

In [427]:
# Initial chromosome generation
def chromosome_generation(number_of_projects, number_of_years):
    matrix = {}
    for i in range(1, number_of_projects + 1):
        matrix[i] = {}
        for j in range(1, number_of_years + 1):
            matrix[i][j] = 0
    for i in range(1, number_of_projects + 1):
        if random.random() <= pst:
            random_column = random.randint(1, number_of_years + 1 - len(c[i]))
            matrix[i][random_column] = 1
    return matrix

In [428]:
# Calculation of reserve
def vypocet_rezervy(b, costs):
    reserve = {}
    reserve[1] = b[1] - costs[1]
    if reserve[1] > 0:
        if reserve[1] >= costs[2]:
            reserve[1] = reserve[1] - costs[2]
            reserve[2] = b[2]
        else:
            reserve[2] = b[2] + reserve[1] - costs[2]
            reserve[1] = 0
    else:
        reserve[2] = b[2] - costs[2]
    for j in range(3, number_of_years + 1):
        if reserve[j - 2] > 0:
            if reserve[j - 2] >= costs[j]:
                reserve[j - 2] = reserve[j - 2] - costs[j]
                reserve[j] = b[j]
            else:
                if reserve[j - 1] > 0:
                    if reserve[j - 1] >= costs[j] - reserve[j - 2]:
                        reserve[j - 1] = reserve[j - 1] + reserve[j - 2] - costs[j]
                        reserve[j - 2] = 0
                        reserve[j] = b[j]
                    else:
                        reserve[j] = b[j] - (costs[j] - (reserve[j - 2] + reserve[j - 1]))
                        reserve[j - 1] = 0
                        reserve[j - 2] = 0
                else:
                    reserve[j] = b[j] + reserve[j - 2] - costs[j]
                    reserve[j - 2] = 0
        else:
            if reserve[j - 1] > 0:
                if reserve[j - 1] >= costs[j]:
                    reserve[j - 1] = reserve[j - 1] - costs[j]
                    reserve[j] = b[j]
                else:
                    reserve[j] = b[j] + reserve[j - 1] - costs[j]
                    reserve[j - 1] = 0
            else:
                reserve[j] = b[j] - costs[j]       
    return reserve

In [429]:
# Matrix C calculation
def vypocet_costs(chromosome):
    output = {i: {j: 0 for j in range(1, number_of_years + 1)} for i in range(1, number_of_projects + 1)}
    for i in chromosome:
        for j in chromosome[i]:
            if chromosome[i][j] == 1:
                for k in range(1, len(c[i]) + 1):
                    output[i][j + k - 1] = c[i][k]
    return output

In [430]:
# Repair operator
def repair_operator(chromosome, c, b):
    #Repair operator for Validity case III
    for i in range(1, number_of_projects + 1):
        if 1 in chromosome[i].values():
            one_index = next((col for col, value in chromosome[i].items() if value == 1), None)
            if int(one_index) > (number_of_years + 1 - len(c[i])):
                chosen_col = random.randint(1, number_of_years + 1 - len(c[i]))
                chromosome[i][one_index] = 0
                chromosome[i][chosen_col] = 1

    # Repair operator for Validity case II
    output = vypocet_costs(chromosome)
    costs = {col: sum(row[col] for row in output.values()) for col in range(1, len(next(iter(output.values()))) + 1)}
    reserve = vypocet_rezervy(b, costs)
    while any(reserve[j] < 0 for j in reserve):
        negative_indices = [index for index, num in reserve.items() if num < 0]
        col = random.choice(negative_indices)          
        ones_indices = [key for key, value in output.items() if value[col] > 0]
        chosen_row = random.choice(ones_indices)
        potential = {}
        for j in range(1, number_of_years + 2 - len(c[chosen_row])):
            potential[j] = True
            soucet_nakladu = 0
            for i in range (1, len(c[chosen_row]) + 1):
                soucet_nakladu += reserve[j + i - 1] - c[chosen_row][i]
                if soucet_nakladu < 0:
                    potential[j] = False
        suitable_columns = [key for key, value in potential.items() if value == True]
        pocatecni_rok = next((col for col, value in chromosome[chosen_row].items() if value == 1), None)
        if suitable_columns == []:
            chromosome[chosen_row][pocatecni_rok] = 0
        else:
            chosen_column = random.choice(suitable_columns)
            chromosome[chosen_row][pocatecni_rok] = 0
            chromosome[chosen_row][chosen_column] = 1
        output = vypocet_costs(chromosome)
        costs = {col: sum(row[col] for row in output.values()) for col in range(1, len(next(iter(output.values()))) + 1)}
        reserve = vypocet_rezervy(b, costs)
    return chromosome

In [431]:
# Fitness calculation
def fitness_evaluation(chromosome, a):
    quality = 0
    for i in range(1, number_of_projects + 1):
        for j in range(1, number_of_years + 1):
            quality += chromosome[i][j] * a[i]
    return quality

In [432]:
# Generation of initial population
def population_generation(population_size):
    population = {}
    for i in range(1, population_size + 1):
        population[i] = {}
        population[i] = chromosome_generation(number_of_projects, number_of_years)
        repair_operator(population[i], c, b)
    return population

In [433]:
# Selection operator
def selection(population, chromosomes_fitness):
    fitness_sum = sum(chromosomes_fitness.values())
    selection_probability = [chromosomes_fitness[i] / fitness_sum for i in range(1, population_size + 1)]
    selection = {}
    for i in range(1, population_size + 1):
        chosen_index = np.random.choice(range(1, population_size + 1), p = selection_probability)
        selection[i] = population[chosen_index]
    set = selection
    return set

In [434]:
# Crossover operator
def krizeni(parent1, parent2):
    if random.random() < crossover_probability:
        crossover_line = random.randint(1, number_of_projects - 1)
        offspring1 = {k: parent1[k] for k in list(parent1)[:crossover_line]} 
        offspring1.update({k: parent2[k] for k in list(parent2)[crossover_line:]})
        offspring2 = {k: parent2[k] for k in list(parent2)[:crossover_line]} 
        offspring2.update({k: parent1[k] for k in list(parent1)[crossover_line:]})
    else:
        offspring1 = parent1
        offspring2 = parent2
    repair_operator(offspring1, c, b)
    repair_operator(offspring2, c, b)
    offsprings = {}
    offsprings[1] = offspring1
    offsprings[2] = offspring2
    return offsprings

In [435]:
# Mutation operator
def mutace(chromosome):
    for j in range(1, number_of_years + 1):
        if random.random() < mutation_probability:
            row = random.choice(list(chromosome.keys()))
            ones_indices = [key for key, value in chromosome[row].items() if value == 1]
            zero_indices = [key for key, value in chromosome[row].items() if value == 0]
            number_of_ones = len(ones_indices)
            if number_of_ones == 0:
                chromosome[row][j] = 1
            else:
                if chromosome[row][j] == 0:
                    one_index = ones_indices[0]
                    chromosome[row][one_index] = 0
                    chromosome[row][j] = 1
                else:
                    chromosome[row][j] = 0
    return chromosome

In [436]:
import pickle
# Genetic algorithm
def geneticky_algoritmus(number_of_projects, number_of_years, a, b, c):
    population = population_generation(population_size)  # Assummationing population_size is defined somewhere
    iteration = 1
    best_solution = repair_operator(chromosome_generation(number_of_projects, number_of_years), c, b)
    summation = {}
    top = {}
    graph_solution = {}
    average_fitness = {}
    start_time = time.time()
    unchanged_fitness = 0

    while iteration <= max_iterations:
        chromosomes_fitness = {}
        for i in range(1, population_size + 1):
            chromosomes_fitness[i] = fitness_evaluation(population[i], a)
        max_value = max(chromosomes_fitness.values())
        best_chromosomes = [key for key, value in chromosomes_fitness.items() if value == max_value]
        best_chromosome = pickle.loads(pickle.dumps(population[random.choice(best_chromosomes)]))
        summation[iteration] = max_value
        top[iteration] = fitness_evaluation(best_solution, a)
        if max_value > fitness_evaluation(best_solution, a):
            best_solution = pickle.loads(pickle.dumps(best_chromosome))
            unchanged_fitness = 0
        else:
            unchanged_fitness += 1
        selection_population = pickle.loads(pickle.dumps(selection(population, chromosomes_fitness)))
        intermediate_population = {}
        for i in range(1, population_size + 1):
            chromosomes_fitness[i] = fitness_evaluation(selection_population[i], a)
        while len(intermediate_population) < population_size:
            set = pickle.loads(pickle.dumps(krizeni(population[random.choice(list(selection_population.keys()))], population[random.choice(list(selection_population.keys()))])))
            intermediate_population[len(intermediate_population) + 1], intermediate_population[len(intermediate_population) + 1] = set[1], set[2]
        for i in range(1, population_size + 1):
            chromosomes_fitness[i] = fitness_evaluation(intermediate_population[i], a)
        pocatek_mutace = time.time()
        for i in range(1, population_size + 1):
            intermediate_population[i] = pickle.loads(pickle.dumps(mutace(intermediate_population[i])))
        for i in range(1, population_size + 1):
            repair_operator(intermediate_population[i], c, b)
            chromosomes_fitness[i] = fitness_evaluation(intermediate_population[i], a)
        population = intermediate_population
        population[random.choice(list(population.keys()))] = pickle.loads(pickle.dumps(best_solution))
        graph_solution[iteration] = fitness_evaluation(best_solution, a)
        average_fitness[iteration] = sum(chromosomes_fitness.values())
        end_time = time.time()
        elapsed_time = end_time - start_time
        iteration += 1
        if unchanged_fitness >= l:
            break
    return population, best_solution, summation, top, graph_solution, average_fitness, elapsed_time

In [437]:
# Input data

b = {1: 24829,
 2: 23729,
 3: 16503,
 4: 15612,
 5: 22222,
 6: 24270,
 7: 17572,
 8: 0,
 9: 0}

a = {1: 43,
 2: 50,
 3: 28,
 4: 25,
 5: 16,
 6: 21,
 7: 31,
 8: 55,
 9: 50,
 10: 56,
 11: 26,
 12: 34,
 13: 28,
 14: 28,
 15: 56,
 16: 10,
 17: 45,
 18: 29,
 19: 37,
 20: 46,
 21: 42,
 22: 57,
 23: 45,
 24: 17,
 25: 5,
 26: 39,
 27: 55,
 28: 17,
 29: 41,
 30: 46,
 31: 32,
 32: 12,
 33: 58,
 34: 53,
 35: 24,
 36: 51,
 37: 10,
 38: 11,
 39: 18,
 40: 45,
 41: 43,
 42: 52,
 43: 59,
 44: 35,
 45: 43,
 46: 58,
 47: 19,
 48: 17,
 49: 53,
 50: 47,
 51: 14,
 52: 48,
 53: 31,
 54: 41,
 55: 53,
 56: 40,
 57: 18,
 58: 10,
 59: 33,
 60: 12,
 61: 33,
 62: 45,
 63: 56,
 64: 6,
 65: 59,
 66: 15,
 67: 20,
 68: 42,
 69: 51,
 70: 29,
 71: 12,
 72: 9,
 73: 39,
 74: 12,
 75: 34,
 76: 44,
 77: 32,
 78: 59,
 79: 6,
 80: 46,
 81: 34,
 82: 50,
 83: 41,
 84: 31,
 85: 29,
 86: 39,
 87: 23,
 88: 50,
 89: 28,
 90: 28,
 91: 24,
 92: 30,
 93: 12,
 94: 14,
 95: 18,
 96: 10,
 97: 9,
 98: 29,
 99: 20,
 100: 21,
 101: 58,
 102: 12,
 103: 11,
 104: 7,
 105: 52,
 106: 39,
 107: 15,
 108: 52,
 109: 9,
 110: 43,
 111: 31,
 112: 16,
 113: 21,
 114: 7,
 115: 11,
 116: 13,
 117: 15,
 118: 24,
 119: 35,
 120: 49,
 121: 27,
 122: 35,
 123: 52,
 124: 53,
 125: 54,
 126: 23,
 127: 9,
 128: 58,
 129: 19,
 130: 43,
 131: 49,
 132: 43,
 133: 32,
 134: 13,
 135: 14,
 136: 48,
 137: 22,
 138: 15,
 139: 6,
 140: 55,
 141: 12,
 142: 21,
 143: 16,
 144: 7,
 145: 33,
 146: 39,
 147: 53,
 148: 60,
 149: 28,
 150: 33,
 151: 57,
 152: 28,
 153: 39,
 154: 25,
 155: 13,
 156: 41,
 157: 36,
 158: 55,
 159: 38,
 160: 47,
 161: 42,
 162: 49,
 163: 40,
 164: 27,
 165: 51,
 166: 16,
 167: 5,
 168: 15,
 169: 11,
 170: 60,
 171: 5,
 172: 34,
 173: 60,
 174: 42,
 175: 11,
 176: 6,
 177: 22,
 178: 34,
 179: 56,
 180: 53,
 181: 12,
 182: 51,
 183: 21,
 184: 18,
 185: 55,
 186: 50,
 187: 13,
 188: 25,
 189: 29,
 190: 9,
 191: 50,
 192: 55,
 193: 15,
 194: 57,
 195: 52,
 196: 17,
 197: 41,
 198: 35,
 199: 50,
 200: 32,
 201: 30,
 202: 49,
 203: 12,
 204: 55,
 205: 17,
 206: 20,
 207: 29,
 208: 39,
 209: 54,
 210: 56,
 211: 50,
 212: 44,
 213: 55,
 214: 27,
 215: 24,
 216: 18,
 217: 49,
 218: 58,
 219: 45,
 220: 12,
 221: 8,
 222: 29,
 223: 6,
 224: 32,
 225: 45,
 226: 29,
 227: 41,
 228: 14,
 229: 33,
 230: 33,
 231: 28,
 232: 45,
 233: 22,
 234: 12,
 235: 59,
 236: 58,
 237: 45,
 238: 29,
 239: 13,
 240: 11,
 241: 18,
 242: 51,
 243: 5,
 244: 9,
 245: 35,
 246: 60,
 247: 58,
 248: 52,
 249: 58,
 250: 38,
 251: 9,
 252: 51,
 253: 13,
 254: 5,
 255: 27,
 256: 52,
 257: 57,
 258: 8,
 259: 8,
 260: 28,
 261: 60,
 262: 53,
 263: 19,
 264: 9,
 265: 25,
 266: 19,
 267: 12,
 268: 33,
 269: 34,
 270: 29,
 271: 52,
 272: 38,
 273: 26,
 274: 57,
 275: 60,
 276: 9,
 277: 37,
 278: 37,
 279: 34,
 280: 55,
 281: 51,
 282: 20,
 283: 56,
 284: 49,
 285: 53,
 286: 25,
 287: 10,
 288: 35,
 289: 32,
 290: 30,
 291: 15,
 292: 60,
 293: 44,
 294: 49,
 295: 12,
 296: 26,
 297: 14,
 298: 48,
 299: 32,
 300: 33}

c = {1: {1: 3641, 2: 4775, 3: 7063},
 2: {1: 1535},
 3: {1: 8841, 2: 3396, 3: 7424, 4: 5773},
 4: {1: 6539, 2: 8995, 3: 4007, 4: 2552, 5: 1234},
 5: {1: 7556, 2: 3960},
 6: {1: 7573},
 7: {1: 9914},
 8: {1: 7042},
 9: {1: 1024, 2: 5604, 3: 7714, 4: 1513},
 10: {1: 7279, 2: 2758, 3: 8999},
 11: {1: 1090, 2: 9472},
 12: {1: 1628, 2: 8060, 3: 9950},
 13: {1: 7899, 2: 1057},
 14: {1: 4881, 2: 1110, 3: 7115, 4: 8324, 5: 7350},
 15: {1: 7531, 2: 4358, 3: 5536, 4: 9246, 5: 7979},
 16: {1: 9745},
 17: {1: 2783, 2: 8205, 3: 6623},
 18: {1: 2405},
 19: {1: 3912, 2: 4429},
 20: {1: 6992, 2: 4915, 3: 3189},
 21: {1: 2560},
 22: {1: 2905, 2: 3969, 3: 6536},
 23: {1: 1315},
 24: {1: 5155, 2: 2360, 3: 4013, 4: 2128},
 25: {1: 8153, 2: 3282, 3: 7793, 4: 6005},
 26: {1: 1346},
 27: {1: 7383},
 28: {1: 9232, 2: 2258, 3: 7486},
 29: {1: 2600},
 30: {1: 2857},
 31: {1: 8549, 2: 5849, 3: 5762},
 32: {1: 9904},
 33: {1: 4458, 2: 8271, 3: 1599},
 34: {1: 5054, 2: 7040},
 35: {1: 4920},
 36: {1: 1100, 2: 8566, 3: 8465},
 37: {1: 8770, 2: 6072, 3: 9758},
 38: {1: 8024, 2: 4396, 3: 3314},
 39: {1: 5946},
 40: {1: 9333, 2: 6352, 3: 7678},
 41: {1: 1818, 2: 7755},
 42: {1: 8084, 2: 6905, 3: 7087},
 43: {1: 1682},
 44: {1: 7144, 2: 1913},
 45: {1: 1213, 2: 9895},
 46: {1: 6973, 2: 6269, 3: 1767},
 47: {1: 4271, 2: 9736, 3: 6307},
 48: {1: 3354},
 49: {1: 8279, 2: 6262},
 50: {1: 4299, 2: 9430},
 51: {1: 2574, 2: 9829, 3: 5322, 4: 5274, 5: 4174},
 52: {1: 4506, 2: 8924, 3: 6179, 4: 6701},
 53: {1: 9041, 2: 8975, 3: 2493, 4: 7419, 5: 2211},
 54: {1: 6187, 2: 3409, 3: 3792, 4: 1738, 5: 7026},
 55: {1: 7021, 2: 2352, 3: 7768},
 56: {1: 1072},
 57: {1: 5289, 2: 1044, 3: 5903, 4: 1268},
 58: {1: 3848, 2: 2908, 3: 3446, 4: 7045},
 59: {1: 4956, 2: 9332, 3: 9933},
 60: {1: 8440},
 61: {1: 8982, 2: 9288, 3: 8077},
 62: {1: 3065},
 63: {1: 8392, 2: 2519, 3: 5001},
 64: {1: 9161, 2: 9077, 3: 8858, 4: 9038, 5: 3324},
 65: {1: 9303, 2: 3742, 3: 8875, 4: 1156, 5: 4073},
 66: {1: 3467, 2: 2701, 3: 7134, 4: 2856, 5: 4440},
 67: {1: 2913},
 68: {1: 4576},
 69: {1: 8078, 2: 9930, 3: 9345},
 70: {1: 8029, 2: 9887, 3: 8841, 4: 1906},
 71: {1: 9947, 2: 8568, 3: 3887, 4: 9140},
 72: {1: 2279},
 73: {1: 6609, 2: 7817, 3: 8581, 4: 4168},
 74: {1: 8070, 2: 4194, 3: 5192, 4: 3263},
 75: {1: 7568, 2: 3998},
 76: {1: 8296, 2: 4627, 3: 6908, 4: 2445, 5: 7037},
 77: {1: 6502, 2: 3611, 3: 5459},
 78: {1: 9043, 2: 9054},
 79: {1: 1435, 2: 7511, 3: 9797, 4: 3282, 5: 2789},
 80: {1: 3462},
 81: {1: 3017, 2: 1880, 3: 5996, 4: 5109},
 82: {1: 6659, 2: 6225, 3: 9724, 4: 4638},
 83: {1: 4443, 2: 9800, 3: 5035},
 84: {1: 1160, 2: 5304, 3: 6759, 4: 8845},
 85: {1: 3260, 2: 7588, 3: 2968, 4: 3398, 5: 8496},
 86: {1: 9576, 2: 4079, 3: 3984, 4: 4779, 5: 8038},
 87: {1: 2478, 2: 1233},
 88: {1: 2624, 2: 5546, 3: 8259},
 89: {1: 7462, 2: 5468, 3: 5264, 4: 1628, 5: 3739},
 90: {1: 8225, 2: 2535, 3: 4561, 4: 3581},
 91: {1: 2098, 2: 8237, 3: 3772},
 92: {1: 1252, 2: 6852, 3: 1033, 4: 6759, 5: 8474},
 93: {1: 2673, 2: 2767, 3: 6439},
 94: {1: 2121, 2: 5486, 3: 9843, 4: 5442, 5: 6286},
 95: {1: 7230, 2: 8655},
 96: {1: 5077, 2: 2548},
 97: {1: 8178, 2: 1603, 3: 1131, 4: 5706, 5: 9241},
 98: {1: 9605},
 99: {1: 1497, 2: 3235},
 100: {1: 1618},
 101: {1: 5320, 2: 3073, 3: 8389, 4: 4052},
 102: {1: 9345, 2: 3885, 3: 3194},
 103: {1: 7235, 2: 2042},
 104: {1: 6128},
 105: {1: 9421, 2: 7690, 3: 7877, 4: 2129},
 106: {1: 1788, 2: 1260, 3: 2171, 4: 4252},
 107: {1: 5109, 2: 4680, 3: 2175, 4: 3557, 5: 6495},
 108: {1: 6268, 2: 6965, 3: 6109, 4: 2346, 5: 3429},
 109: {1: 8032, 2: 4635},
 110: {1: 1394, 2: 9365, 3: 8384, 4: 6976, 5: 7129},
 111: {1: 6339, 2: 7012, 3: 8183},
 112: {1: 5650, 2: 5276, 3: 5485, 4: 3990, 5: 9811},
 113: {1: 6428, 2: 5637, 3: 8701, 4: 8172},
 114: {1: 2871, 2: 6927, 3: 8501},
 115: {1: 9688, 2: 1811, 3: 5054, 4: 9985, 5: 6924},
 116: {1: 9461},
 117: {1: 8673, 2: 1052},
 118: {1: 3109, 2: 1569, 3: 1027, 4: 6581, 5: 5222},
 119: {1: 7932, 2: 8331, 3: 9652, 4: 4011},
 120: {1: 7649},
 121: {1: 3635, 2: 2835},
 122: {1: 8391},
 123: {1: 4314, 2: 8699, 3: 6047},
 124: {1: 1994},
 125: {1: 5611, 2: 3118, 3: 9604, 4: 3999},
 126: {1: 2426, 2: 6950},
 127: {1: 8015, 2: 1711, 3: 7373, 4: 5953, 5: 6324},
 128: {1: 9206, 2: 6587, 3: 6433},
 129: {1: 6235},
 130: {1: 1409, 2: 3838, 3: 2465, 4: 5455, 5: 6322},
 131: {1: 5278, 2: 2636, 3: 6515},
 132: {1: 1066, 2: 4748},
 133: {1: 6983},
 134: {1: 6320, 2: 5946, 3: 8204, 4: 9662},
 135: {1: 9969, 2: 1915, 3: 8480},
 136: {1: 6452, 2: 9670},
 137: {1: 8549, 2: 6641},
 138: {1: 8473, 2: 2362},
 139: {1: 7925, 2: 9949, 3: 8091},
 140: {1: 5787, 2: 8769},
 141: {1: 4148},
 142: {1: 5955, 2: 9994, 3: 1962, 4: 3761, 5: 2383},
 143: {1: 2054, 2: 3007, 3: 7793},
 144: {1: 5241},
 145: {1: 1379, 2: 9046, 3: 3739, 4: 5170},
 146: {1: 7185},
 147: {1: 3151, 2: 5115, 3: 3077, 4: 5927, 5: 6094},
 148: {1: 3951, 2: 4048, 3: 1433, 4: 1854, 5: 3807},
 149: {1: 1105, 2: 9998, 3: 4328, 4: 2890, 5: 5591},
 150: {1: 5835, 2: 9137, 3: 7129, 4: 2914, 5: 5935},
 151: {1: 1765, 2: 7428},
 152: {1: 2482},
 153: {1: 4473, 2: 3860, 3: 5340},
 154: {1: 5764, 2: 5500, 3: 8936},
 155: {1: 3967},
 156: {1: 7868, 2: 3718, 3: 9359, 4: 8256, 5: 4128},
 157: {1: 3818},
 158: {1: 9066},
 159: {1: 1283, 2: 4379, 3: 5164, 4: 7569, 5: 7839},
 160: {1: 3249, 2: 4889},
 161: {1: 9080, 2: 5036, 3: 5997, 4: 8189},
 162: {1: 4809, 2: 6938, 3: 8429, 4: 8343},
 163: {1: 7667, 2: 3693},
 164: {1: 8978, 2: 5474, 3: 8179, 4: 2248, 5: 1831},
 165: {1: 1766, 2: 5426, 3: 8633, 4: 2747},
 166: {1: 5166, 2: 7165, 3: 9216, 4: 2222},
 167: {1: 8283, 2: 5712, 3: 1288},
 168: {1: 1099},
 169: {1: 8699, 2: 8498},
 170: {1: 8017, 2: 2273, 3: 6463},
 171: {1: 5384, 2: 8216, 3: 7829, 4: 6158},
 172: {1: 7848, 2: 3278, 3: 7377},
 173: {1: 7124, 2: 5325},
 174: {1: 5116, 2: 3106, 3: 2694, 4: 4954},
 175: {1: 7614, 2: 2524, 3: 6095, 4: 6318, 5: 2574},
 176: {1: 8777, 2: 6607, 3: 6196, 4: 9803},
 177: {1: 3064, 2: 6226, 3: 4415, 4: 9923, 5: 4937},
 178: {1: 3292, 2: 2900, 3: 6824},
 179: {1: 2523},
 180: {1: 5913, 2: 5478, 3: 1614, 4: 1101},
 181: {1: 1676, 2: 9489, 3: 9497, 4: 2333},
 182: {1: 6315},
 183: {1: 3980, 2: 9371, 3: 9363},
 184: {1: 7110, 2: 7095},
 185: {1: 3212, 2: 3304, 3: 4678},
 186: {1: 4782, 2: 7082, 3: 8805},
 187: {1: 3381},
 188: {1: 1372, 2: 3904, 3: 4993, 4: 2151, 5: 8106},
 189: {1: 3853, 2: 2797, 3: 7920, 4: 7758, 5: 8979},
 190: {1: 7353, 2: 2360, 3: 8629},
 191: {1: 8708, 2: 6702, 3: 1579},
 192: {1: 6864, 2: 2670, 3: 2025, 4: 1271},
 193: {1: 5883, 2: 5227, 3: 4100, 4: 6234},
 194: {1: 8028, 2: 7806},
 195: {1: 8783, 2: 6332, 3: 4011},
 196: {1: 1728, 2: 2108, 3: 7782, 4: 2016},
 197: {1: 9077},
 198: {1: 4566, 2: 8436, 3: 8874},
 199: {1: 3874, 2: 5899},
 200: {1: 1843, 2: 7433},
 201: {1: 3647},
 202: {1: 8434},
 203: {1: 1045},
 204: {1: 3993},
 205: {1: 7902, 2: 1761, 3: 5307, 4: 4645, 5: 8495},
 206: {1: 7810, 2: 7583},
 207: {1: 2058, 2: 2004, 3: 1607},
 208: {1: 9973, 2: 9137, 3: 3733, 4: 4820},
 209: {1: 1378, 2: 9971},
 210: {1: 4598},
 211: {1: 6702},
 212: {1: 8094, 2: 6409},
 213: {1: 1354, 2: 5233},
 214: {1: 4535, 2: 8385, 3: 7705},
 215: {1: 7494},
 216: {1: 6616, 2: 1620, 3: 2346, 4: 2908, 5: 8626},
 217: {1: 1096, 2: 7259},
 218: {1: 8441, 2: 6930, 3: 5504, 4: 3367},
 219: {1: 6103},
 220: {1: 2527, 2: 1427, 3: 5132, 4: 6096},
 221: {1: 2457, 2: 3107},
 222: {1: 7328, 2: 6460, 3: 5930, 4: 6066},
 223: {1: 7611},
 224: {1: 4219, 2: 8590},
 225: {1: 5439},
 226: {1: 6386},
 227: {1: 4688, 2: 1416, 3: 4971, 4: 5926, 5: 3714},
 228: {1: 2466, 2: 8696, 3: 7322, 4: 7189},
 229: {1: 5174, 2: 1847, 3: 2143, 4: 7099},
 230: {1: 7146, 2: 3410, 3: 1033, 4: 4606, 5: 5240},
 231: {1: 3670, 2: 9747, 3: 8892, 4: 4225, 5: 3546},
 232: {1: 7700},
 233: {1: 6280, 2: 5888},
 234: {1: 4330, 2: 5382, 3: 3522, 4: 8385, 5: 3179},
 235: {1: 2470, 2: 9712, 3: 9302, 4: 6225},
 236: {1: 7529, 2: 6718, 3: 8322},
 237: {1: 9681, 2: 9059, 3: 8561, 4: 9945},
 238: {1: 7617, 2: 2011, 3: 2301, 4: 9929, 5: 1013},
 239: {1: 1224, 2: 2781, 3: 6170},
 240: {1: 1800, 2: 3112},
 241: {1: 2029, 2: 5169, 3: 6882, 4: 5602, 5: 3815},
 242: {1: 6481},
 243: {1: 4544, 2: 3679, 3: 6541, 4: 1766, 5: 6482},
 244: {1: 6747, 2: 8638, 3: 7279, 4: 1053, 5: 6846},
 245: {1: 2347, 2: 3695, 3: 2883},
 246: {1: 6265, 2: 7273, 3: 1209, 4: 4731, 5: 2415},
 247: {1: 7603, 2: 4640},
 248: {1: 2361, 2: 7465, 3: 2716},
 249: {1: 3303, 2: 5541},
 250: {1: 9151, 2: 9671},
 251: {1: 3107, 2: 9056, 3: 7892, 4: 4417},
 252: {1: 8242, 2: 6382, 3: 9654, 4: 2903},
 253: {1: 2557, 2: 4225, 3: 6887, 4: 4827, 5: 9110},
 254: {1: 1322, 2: 5612, 3: 7736},
 255: {1: 9975},
 256: {1: 8705},
 257: {1: 5386, 2: 2959, 3: 8994, 4: 7779, 5: 8449},
 258: {1: 8132, 2: 6409},
 259: {1: 2910, 2: 5316, 3: 5015, 4: 9837, 5: 5007},
 260: {1: 8515, 2: 7616},
 261: {1: 2355, 2: 6100, 3: 5085},
 262: {1: 9009, 2: 7446},
 263: {1: 6439, 2: 5768, 3: 9211},
 264: {1: 4078, 2: 4020, 3: 6694, 4: 5383, 5: 1740},
 265: {1: 7335, 2: 9307},
 266: {1: 5335, 2: 2756, 3: 8122, 4: 7843},
 267: {1: 6814, 2: 7020, 3: 2806},
 268: {1: 4408, 2: 5560, 3: 6774},
 269: {1: 6118, 2: 1549},
 270: {1: 9154, 2: 7958},
 271: {1: 6538, 2: 8489, 3: 9498, 4: 4013},
 272: {1: 2128, 2: 9749, 3: 7886, 4: 2322},
 273: {1: 7307, 2: 2011, 3: 1935, 4: 3116, 5: 9511},
 274: {1: 9681, 2: 1544, 3: 2152, 4: 5854},
 275: {1: 1500, 2: 7885},
 276: {1: 2365, 2: 7176, 3: 4117},
 277: {1: 1983, 2: 2284},
 278: {1: 1369, 2: 7463},
 279: {1: 7005},
 280: {1: 2880, 2: 6753, 3: 7708, 4: 6474},
 281: {1: 3545, 2: 4163},
 282: {1: 8950, 2: 1692},
 283: {1: 8283, 2: 8819, 3: 5999, 4: 6617, 5: 9009},
 284: {1: 9174, 2: 7038, 3: 1535, 4: 6501, 5: 5332},
 285: {1: 7413},
 286: {1: 6820, 2: 1833, 3: 5652, 4: 6677, 5: 3478},
 287: {1: 3347, 2: 4951},
 288: {1: 5690, 2: 2820},
 289: {1: 4837, 2: 6735, 3: 6705, 4: 9199, 5: 3300},
 290: {1: 3886, 2: 2545, 3: 1957, 4: 9347},
 291: {1: 7881, 2: 8630, 3: 6461},
 292: {1: 6382, 2: 7115, 3: 6750, 4: 3163, 5: 3561},
 293: {1: 4360, 2: 6582, 3: 8715, 4: 1740, 5: 5476},
 294: {1: 6650, 2: 9125},
 295: {1: 6112},
 296: {1: 3466, 2: 2557, 3: 2757},
 297: {1: 2139, 2: 9658, 3: 6152, 4: 2271},
 298: {1: 4843, 2: 7544, 3: 7783, 4: 5616, 5: 4390},
 299: {1: 3196, 2: 6671},
 300: {1: 7091, 2: 7818, 3: 4278, 4: 5578}}

In [438]:
# Calculation 

number_of_projects = 300
number_of_years = 9

c_sum = 0
for outer_key in c:
    inner_dict = c[outer_key]
    for inner_key in inner_dict:
        c_sum += inner_dict[inner_key]

# Nastavení parametrů GA
pst = sum(b.values()) / c_sum

population_size = 10
max_iterations = 5000
crossover_probability = 0.9
mutation_probability = 0.056
l = 500

df_columns = []
df_results = []
df_columns = ['Maximum number of iterations', 'Crossover probability', 'Mutation probability', 'Population size', 'Max number of interations with unchanged best solution', 'Real number of iterations', 'Computational time', 'Fitness value']
df_results = pd.DataFrame(columns = df_columns)


for p in range(1, 11):
    population, nejlepsi, summation, top, graf, kvalita, elapsed_time = geneticky_algoritmus(number_of_projects, number_of_years, a, b, c)
    new_row = pd.DataFrame({
        'Maximum number of iterations': [max_iterations],
        'Crossover probability': [crossover_probability],
        'Mutation probability': [mutation_probability],
        'Max number of interations with unchanged best solution': [l],
        'Population size': [population_size],
        'Real number of iterations': [max(top)],
        'Computational time': [elapsed_time],
        'Fitness value': [max(top.values())]
    })
    df_results = pd.concat([df_results, new_row], ignore_index=True)
df_results.index.name = 'GA run'

In [439]:
df_results

,Maximum number of iterations,Crossover probability,Mutation probability,Population size,Max number of interations with unchanged best solution,Real number of iterations,Computational time,Fitness value
GA run,,,,,,,,
0,5000,0.9,0.056,10,500,3451,144.111728,1467
1,5000,0.9,0.056,10,500,4157,182.038630,1416
2,5000,0.9,0.056,10,500,2794,121.211836,1434
3,5000,0.9,0.056,10,500,3485,112.428647,1435
4,5000,0.9,0.056,10,500,1921,63.551827,1385
5,5000,0.9,0.056,10,500,1676,73.134410,1350
6,5000,0.9,0.056,10,500,4119,120.873960,1416
7,5000,0.9,0.056,10,500,2316,64.922788,1388
8,5000,0.9,0.056,10,500,2688,81.236602,1349


In [440]:
df_results['Mutation probability'] = df_results['Mutation probability'].round(3)

In [441]:
df_results.to_excel('Results_overall.xlsx')